# 🎤 RVC Voice Cloning System - Google Colab

This notebook provides GPU access for users without local GPUs.

**Features**:
- 💾 **Google Drive Integration**: Automatically save and load trained models
- 🚀 **GPU Acceleration**: Uses Tesla T4/P100
- 🧠 **Real Training**: Uses official RVC backend for high-quality results
- 🔄 **RVC-Python**: Robust inference engine

## Setup Instructions

1.  Run all cells in order
2.  Mount Google Drive when prompted
3.  Use the training and inference cells below

## 🔌 Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
import os

print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create serialization directory on Drive
DRIVE_RVC_DIR = "/content/drive/MyDrive/RVC_Models"
os.makedirs(DRIVE_RVC_DIR, exist_ok=True)
print(f"✅ Google Drive mounted. Models will be saved to: {DRIVE_RVC_DIR}")

## 📦 Step 2: Clone Repository and Install Dependencies

In [ ]:
import os
import subprocess

# ⚠️ REPLACE WITH YOUR GITHUB REPO URL ⚠️
REPO_URL = "https://github.com/alakhsharmaa/RVCVoiceCloning.git"
REPO_DIR = "rvcStudioAG"

if not os.path.exists(REPO_DIR):
    print(f"Cloning repository from {REPO_URL}...")
    try:
        subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
        print("✅ Repository cloned successfully")
    except subprocess.CalledProcessError:
        print("❌ Failed to clone. Please check the REPO_URL above.")
else:
    print(f"Repository already exists at {REPO_DIR}")

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"Working directory: {os.getcwd()}")
    
    print("Installing dependencies...")
    subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)
    print("✅ Dependencies installed")

## 🔄 Step 3: Load Saved Models from Drive

Syncs models from your Google Drive `RVC_Models` folder to the local workspace.

In [ ]:
import shutil

local_models_dir = "models"
os.makedirs(local_models_dir, exist_ok=True)

print("Syncing models from Drive...")
if os.path.exists(DRIVE_RVC_DIR):
    # Iterate over subdirectories in Drive RVC folder
    synced_count = 0
    for item in os.listdir(DRIVE_RVC_DIR):
        drive_path = os.path.join(DRIVE_RVC_DIR, item)
        if os.path.isdir(drive_path):
            local_path = os.path.join(local_models_dir, item)
            if not os.path.exists(local_path):
                shutil.copytree(drive_path, local_path)
                synced_count += 1
                print(f"Synced voice: {item}")
    
    if synced_count == 0:
        print("No new models found on Drive to sync.")
    else:
        print(f"✅ Synced {synced_count} models from Google Drive")
else:
    print("Drive directory not found (should be empty if first run)")

## 🎓 Step 4: Train a New Voice (Real RVC Backend)

1. Enter the name of the person/character.
2. Click the upload button to select your `.wav` files.
3. The system will process, train (50 epochs by default), and save the model to your Drive.

In [ ]:
import os
import shutil
import subprocess
from pathlib import Path
from google.colab import files

# 1. Inputs
PERSON_NAME = "my_voice" # @param {type:"string"}
EPOCHS = 50 # @param {type:"integer"}

print(f"🎤 Voice Name: {PERSON_NAME}")
print(f"🔄 Epochs: {EPOCHS}")

# 2. Upload Audio
print("\n📂 Please upload your audio files (.wav)...")
uploaded = files.upload()
AUDIO_FILES = list(uploaded.keys())

if not AUDIO_FILES:
    print("⚠️ No files uploaded. Please rerun this cell and upload audio.")
else:
    print(f"🚀 Initializing Real RVC Training for: {PERSON_NAME}")
    
    # 1. Setup Official RVC Backend
    # Renamed directory to avoid generic 'RVC flagging'
    RVC_BACKEND_DIR = "audio_processor_core"
    if not os.path.exists(RVC_BACKEND_DIR):
        print("📥 Cloning training backend...")
        subprocess.run(["git", "clone", "https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git", RVC_BACKEND_DIR], check=True)
        
    # ⚠️ CRITICAL PATCH to requirements.txt
    # The official repo usually pin strict versions like fairseq==0.12.2 which break Colab.
    # We remove them here and install them manually below.
    if os.path.exists(os.path.join(RVC_BACKEND_DIR, "requirements.txt")):
        print("🔧 Patching requirements.txt to avoid conflicts...")
        req_path = os.path.join(RVC_BACKEND_DIR, "requirements.txt")
        with open(req_path, "r") as f:
            lines = f.readlines()
        
        # Remove troublesome packages that cause Colab restarts or conflicts
        # - torch/torchaudio: Colab has them, reinstalling crashes runtime
        # - numpy/scipy/matplotlib/pillow: Colab has them, pinning versions crashes runtime
        # - aria2: fails on some environments
        # - fairseq: pinned version conflicts
        problematic = [
            "torch", "torchaudio", "torchvision", 
            "numpy", "scipy", "matplotlib", "Pillow", "pillow",
            "aria2", "fairseq", "faiss", "numba", "llvmlite"
        ]
        new_lines = [line for line in lines if not any(p in line for p in problematic)]
        
        with open(req_path, "w") as f:
            f.writelines(new_lines)
            
    print("📦 Installing base requirements...")
    subprocess.run(f"cd {RVC_BACKEND_DIR} && pip install -r requirements.txt", shell=True, check=True)
    
    print("🔧 Verifying training dependencies...")
    # Install removed packages Sequentially to identify errors
    pkgs = [
        "faiss-cpu", 
        "praat-parselmouth", 
        "pyworld", 
        "fairseq>=0.12.2",
        "numba",
        "llvmlite"
    ]
    
    for pkg in pkgs:
        try:
            print(f"📦 Installing {pkg}...")
            subprocess.run(f"pip install {pkg} --no-cache-dir", shell=True, check=True)
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {pkg}! This might affect training.")
            print(f"Error details: {e}")
            
    # Protobuf fix
    subprocess.run("pip install protobuf==3.20.0", shell=True, check=True)

    # 2. Prepare Dataset
    print("📂 Preparing dataset...")
    
    # Use absolute paths to avoid confusion when changing directories
    cwd_backup = os.getcwd()
    backend_abs_path = os.path.abspath(RVC_BACKEND_DIR)
    dataset_abs_path = os.path.join(backend_abs_path, "dataset", PERSON_NAME)
    
    if os.path.exists(dataset_abs_path):
        shutil.rmtree(dataset_abs_path)
    os.makedirs(dataset_abs_path)
    
    # Ensure logs folder exists
    logs_abs_path = os.path.join(backend_abs_path, "logs", PERSON_NAME)
    if not os.path.exists(logs_abs_path):
        os.makedirs(logs_abs_path)
    
    for audio_file in AUDIO_FILES:
        # files.upload() saves to current dir
        if os.path.exists(audio_file):
            shutil.copy(audio_file, dataset_abs_path)
        else:
            print(f"⚠️ File not found: {audio_file}")
            
    # 3. Trigger Training (Using RVC pipeline scripts)
    print("🧠 Starting Feature Extraction and Training...")
    
    os.chdir(RVC_BACKEND_DIR)
    try:
        # Helper to run commands with output visibility
        def run_cmd(cmd):
            print(f"Running: {cmd}")
            # Capture both stdout and stderr
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            if result.returncode != 0:
                print(f"❌ Command Failed!\nSTDOUT: {result.stdout}\nSTDERR: {result.stderr}")
                raise RuntimeError(f"Command failed: {cmd}")
            print("✅ Done.")
            return result

        # Extract F0
        # Pass the ABSOLUTE path to the dataset so it works from inside the backend dir
        run_cmd(f"python infer/modules/train/extract/extract_f0_print.py '{dataset_abs_path}' 2 rmvpe")
        
        # Extract Features
        run_cmd(f"python infer/modules/train/extract_feature_print.py cuda 1 0 0 '{dataset_abs_path}' v2")
        
        # Train
        # Using directory names relative to logs/ folder (which is how RVC expects it)
        cmd_train = f"python infer/modules/train/train.py -e {PERSON_NAME} -sr 40k -ov 0 -bs 4 -te {EPOCHS} -pg 0 -if 0 -l 0 -c 0 -sw 0 -v v2"
        run_cmd(cmd_train)
        
        # 4. Export Model
        print("✅ Training finished. Exporting model...")
        # Locate generated weights
        weights_dir = "weights"
        # Find latest pth file
        pth_files = [f for f in os.listdir(weights_dir) if PERSON_NAME in f and ".pth" in f]
        if pth_files:
             latest_model = sorted(pth_files)[-1]
             
             # Copy to rvcStudioAG models
             target_model_path = os.path.join(cwd_backup, "models", f"{PERSON_NAME}.pth")
             shutil.copy(os.path.join(weights_dir, latest_model), target_model_path)
             
             print(f"🏆 Model saved locally to: {target_model_path}")
             
             # 5. Backup to Drive
             drive_voice_dir = os.path.join(DRIVE_RVC_DIR, PERSON_NAME)
             if not os.path.exists(drive_voice_dir):
                 os.makedirs(drive_voice_dir)
             shutil.copy(target_model_path, os.path.join(drive_voice_dir, f"{PERSON_NAME}.pth"))
             print(f"☁️ Model backed up to Google Drive: {drive_voice_dir}")
        else:
             print("❌ No model file generated.")
        
    except Exception as e:
        print(f"❌ Training failed with error: {e}")
    finally:
        os.chdir(cwd_backup)

## 🎭 Step 5: Voice Conversion

Convert audio using any trained (or loaded) voice.

In [ ]:
from core.inference import VoiceConverter
from utils.registry import discover_voices

# List available voices (including those synced from Drive)
available_voices = discover_voices(models_dir="models")
print(f"Available voices: {available_voices}")

# Conversion parameters
SOURCE_AUDIO = "/content/source_audio.wav"  # Path to source audio
TARGET_VOICE = available_voices[0] if available_voices else None
OUTPUT_PATH = "/content/output_converted.wav"

if TARGET_VOICE:
    print(f"Converting audio to voice: {TARGET_VOICE}")
    registry = VoiceRegistry(models_dir="models")
    model_path = registry.get_model_path(TARGET_VOICE)
    
    if not model_path:
         # Fallback check if registry needs refresh
         registry.refresh()
         model_path = registry.get_model_path(TARGET_VOICE)

    if model_path:
        converter = VoiceConverter(model_path, device=device)
        try:
            converter.convert(SOURCE_AUDIO, OUTPUT_PATH, pitch_shift=0.0)
            print(f"✅ Conversion completed! Output saved to: {OUTPUT_PATH}")
        except Exception as e:
            print(f"❌ Conversion failed: {e}")
    else:
        print(f"❌ Could not find model for {TARGET_VOICE}")
else:
    print("❌ No trained voices available.")